# Query Analyzer Agent Test

This notebook implements and tests the Query Analyzer Agent that understands natural language queries, enriches them using LLM's built-in domain knowledge, and decomposes complex queries into logical parts.

In [1]:
from dotenv import load_dotenv
import json
import re
from typing import Dict, Any, List, Optional
import uuid

load_dotenv()

# Import unified schemas from our centralized location
from schemas import (
    # Query Analyzer types
    QueryAnalysisInput,
    QueryAnalysisOutput,
    QueryPlan,
    QueryPart,
    ExtractedEntitiesAndIntent,
    
    # Error types
    QueryAnalysisError,
    
    # Database schemas
    SCHEMAS,
    QUERY_PATTERNS
)

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from schema_manager import SchemaManager

## Define Query Analyzer System Prompts

In [ ]:
# Single unified system prompt for query analysis and decomposition
QUERY_ANALYZER_SYSTEM_PROMPT = """You are an expert Query Analyzer Agent that understands natural language queries about databases.

You have two main tasks:

1. Query Analysis:
   When analyzing queries, return results in this XML format:

   <analysis>
     <enriched_query>Enriched natural language query with domain knowledge applied</enriched_query>
     
     <complexity_analysis>
       <is_complex>true/false</is_complex>
       <complexity_factors>
         <factor>Factor description</factor>
         ...
       </complexity_factors>
       <confidence>0.0-1.0</confidence>
     </complexity_analysis>
     
     <entity_extraction>
       <metrics>
         <metric>metric_name</metric>
         ...
       </metrics>
       <dimensions>
         <dimension>dimension_name</dimension>
         ...
       </dimensions>
       <filters>
         <filter>
           <field>field_name</field>
           <operator>operator</operator>
           <value>value</value>
         </filter>
         ...
       </filters>
       <primary_goal>goal_type</primary_goal>
       <confidence>0.0-1.0</confidence>
     </entity_extraction>
   </analysis>

   When enriching queries:
   - Understand common synonyms and business terminology from the query context
   - Apply logical business rules based on the query intent
   - Identify database entities being referenced
   - Use your knowledge of database concepts and SQL patterns

   For entity extraction, identify:
   - Entities: What table and fields are queried
   - Metrics: What is being measured (count, sum, average, etc.)
   - Dimensions: What groupings or categories are involved
   - Filters: What conditions limit the data
   - Primary goal: The main purpose (aggregation, comparison, ranking, etc.)

   Complex queries include:
   - Comparisons with calculated values (e.g., "above average")
   - Multi-step analyses (e.g., find max then get details)
   - Queries combining multiple unrelated questions
   - Nested conditions that require intermediate results

2. Query Decomposition:
   When decomposing complex queries, return results in this XML format:

   <decomposition>
     <parts>
       <part>
         <part_id>unique_id</part_id>
         <sub_query>Natural language sub-query</sub_query>
         <metrics>
           <metric>metric_name</metric>
           ...
         </metrics>
         <dimensions>
           <dimension>dimension_name</dimension>
           ...
         </dimensions>
         <filters>
           <filter>
             <field>field_name</field>
             <operator>operator</operator>
             <value>value</value>
           </filter>
           ...
         </filters>
         <primary_goal>goal_type</primary_goal>
         <dependencies>
           <dependency>part_id</dependency>
           ...
         </dependencies>
       </part>
       ...
     </parts>
     <assembly_instructions>Instructions for combining results</assembly_instructions>
   </decomposition>

   Each part should be a complete, executable sub-query.
   Identify dependencies where one part's results are needed for another.
   Provide clear assembly instructions for combining the results.

IMPORTANT: Always respond in the appropriate XML format based on the task requested."""

In [ ]:
import asyncio
from dataclasses import asdict
import logging
import xml.etree.ElementTree as ET
import re
import traceback
import json

logger = logging.getLogger(__name__)

class QueryAnalyzerAgent:
    """Agent that analyzes natural language queries and creates execution plans."""
    
    def __init__(self, config: Optional[Dict] = None):
        """Initialize the Query Analyzer Agent."""
        self.config = config or {}
        self.model = self.config.get('model', 'gpt-4o')
        self.model_client = OpenAIChatCompletionClient(model=self.model)
        self.agent = self._create_agent()
        
    def _create_agent(self) -> AssistantAgent:
        """Create the query analyzer agent."""
        
        # Define deterministic tools that process the agent's responses
        async def parse_analysis_xml(xml_response: str) -> str:
            """Parse XML analysis response from the agent."""
            try:
                xml_match = re.search(r'<analysis>.*?</analysis>', xml_response, re.DOTALL)
                if xml_match:
                    xml_content = xml_match.group()
                    root = ET.fromstring(xml_content)
                    
                    result = {}
                    
                    # Extract enriched query
                    enriched_elem = root.find('enriched_query')
                    if enriched_elem is not None and enriched_elem.text:
                        result["enriched_query"] = enriched_elem.text.strip()
                    
                    # Extract complexity analysis
                    complexity_elem = root.find('complexity_analysis')
                    if complexity_elem is not None:
                        is_complex_elem = complexity_elem.find('is_complex')
                        confidence_elem = complexity_elem.find('confidence')
                        
                        result["complexity_analysis"] = {
                            "is_complex": is_complex_elem.text.lower() == 'true' if is_complex_elem is not None else False,
                            "confidence": float(confidence_elem.text) if confidence_elem is not None else 0.5,
                            "complexity_factors": [f.text for f in complexity_elem.findall('.//factor') if f.text]
                        }
                    
                    # Extract entities
                    entity_elem = root.find('entity_extraction')
                    if entity_elem is not None:
                        metrics = [m.text for m in entity_elem.findall('.//metric') if m.text]
                        dimensions = [d.text for d in entity_elem.findall('.//dimension') if d.text]
                        
                        filters = []
                        for filter_elem in entity_elem.findall('.//filter'):
                            field_elem = filter_elem.find('field')
                            operator_elem = filter_elem.find('operator')
                            value_elem = filter_elem.find('value')
                            
                            if field_elem is not None and operator_elem is not None and value_elem is not None:
                                filters.append({
                                    "field": field_elem.text,
                                    "operator": operator_elem.text,
                                    "value": value_elem.text
                                })
                        
                        primary_goal_elem = entity_elem.find('primary_goal')
                        confidence_elem = entity_elem.find('confidence')
                        
                        result["entity_extraction"] = {
                            "metrics": metrics,
                            "dimensions": dimensions,
                            "filters": filters,
                            "primary_goal": primary_goal_elem.text if primary_goal_elem is not None else "retrieve",
                            "confidence": float(confidence_elem.text) if confidence_elem is not None else 0.5
                        }
                    
                    return json.dumps(result, indent=2)
                else:
                    raise ValueError("No valid XML analysis found in response")
                    
            except Exception as e:
                logger.error(f"Failed to parse XML analysis: {str(e)}")
                # Return minimal result on failure
                default_result = {
                    "enriched_query": "",
                    "complexity_analysis": {
                        "is_complex": False,
                        "confidence": 0.5
                    },
                    "entity_extraction": {
                        "metrics": ["data"],
                        "dimensions": [],
                        "filters": [],
                        "primary_goal": "retrieve",
                        "confidence": 0.5
                    }
                }
                return json.dumps(default_result, indent=2)
        
        async def parse_decomposition_xml(xml_response: str) -> str:
            """Parse XML decomposition response from the agent."""
            try:
                xml_match = re.search(r'<decomposition>.*?</decomposition>', xml_response, re.DOTALL)
                if xml_match:
                    xml_content = xml_match.group()
                    root = ET.fromstring(xml_content)
                    
                    result = {
                        "parts": [],
                        "assembly_instructions": ""
                    }
                    
                    # Extract parts
                    parts_elem = root.find('parts')
                    if parts_elem is not None:
                        for part_elem in parts_elem.findall('part'):
                            part_id_elem = part_elem.find('part_id')
                            sub_query_elem = part_elem.find('sub_query')
                            
                            if part_id_elem is not None and sub_query_elem is not None:
                                part_data = {
                                    "part_id": part_id_elem.text,
                                    "sub_query": sub_query_elem.text,
                                    "metrics": [m.text for m in part_elem.findall('.//metric') if m.text],
                                    "dimensions": [d.text for d in part_elem.findall('.//dimension') if d.text],
                                    "filters": [],
                                    "dependencies": [dep.text for dep in part_elem.findall('.//dependency') if dep.text],
                                    "primary_goal": "retrieve"
                                }
                                
                                # Extract filters
                                for filter_elem in part_elem.findall('.//filter'):
                                    field_elem = filter_elem.find('field')
                                    operator_elem = filter_elem.find('operator')
                                    value_elem = filter_elem.find('value')
                                    
                                    if all([field_elem is not None, operator_elem is not None, value_elem is not None]):
                                        part_data["filters"].append({
                                            "field": field_elem.text,
                                            "operator": operator_elem.text,
                                            "value": value_elem.text
                                        })
                                
                                primary_goal_elem = part_elem.find('primary_goal')
                                if primary_goal_elem is not None:
                                    part_data["primary_goal"] = primary_goal_elem.text
                                
                                result["parts"].append(part_data)
                    
                    # Extract assembly instructions
                    assembly_elem = root.find('assembly_instructions')
                    if assembly_elem is not None and assembly_elem.text:
                        result["assembly_instructions"] = assembly_elem.text.strip()
                    
                    return json.dumps(result, indent=2)
                else:
                    raise ValueError("No valid XML decomposition found in response")
                    
            except Exception as e:
                logger.error(f"Failed to parse XML decomposition: {str(e)}")
                default_result = {
                    "parts": [],
                    "assembly_instructions": ""
                }
                return json.dumps(default_result, indent=2)
        
        # Create the agent with deterministic parsing tools
        return AssistantAgent(
            name="query_analyzer",
            model_client=self.model_client,
            tools=[parse_analysis_xml, parse_decomposition_xml],
            system_message=QUERY_ANALYZER_SYSTEM_PROMPT,
            reflect_on_tool_use=True,
            model_client_stream=True,
        )
    
    async def analyze(self, input_data: QueryAnalysisInput) -> QueryAnalysisOutput:
        """
        Main entry point for query analysis.
        
        Args:
            input_data: QueryAnalysisInput containing the query and context
            
        Returns:
            QueryAnalysisOutput with analysis results
            
        Raises:
            QueryAnalysisError: If analysis fails
        """
        try:
            # First LLM call - perform initial analysis
            analysis_prompt = f"""
            Analyze this query:
            Query: "{input_data.natural_language_query}"
            Database ID: {input_data.database_id}
            
            Perform the following tasks:
            1. Enrich the query with your understanding of database concepts and SQL patterns
            2. Analyze query complexity (is it multi-step, has comparisons, etc.)
            3. Extract entities and intent (metrics, dimensions, filters, primary goal)
            
            Use your knowledge of common database terminology, business rules, and SQL patterns to understand the query intent.
            
            Return analysis in XML format as specified in the system prompt.
            """
            
            if input_data.database_schema:
                analysis_prompt += f"\n\nDatabase Schema:\n{input_data.database_schema}"
            
            # Run the agent to get XML analysis
            analysis_result_str = await self.agent.arun(task=analysis_prompt)
            
            # Use the parsing tool to extract structured data
            parsed_analysis_str = await self.agent.arun(
                task=f"Parse this XML analysis response:\n{analysis_result_str}"
            )
            analysis_result = json.loads(parsed_analysis_str)
            
            # Extract results
            enriched_query = analysis_result.get("enriched_query", input_data.natural_language_query)
            complexity_analysis = analysis_result.get("complexity_analysis", {
                "is_complex": False,
                "confidence": 0.5
            })
            main_entities = analysis_result.get("entity_extraction", {})
            
            # Create ExtractedEntitiesAndIntent object
            main_entities_and_intent = ExtractedEntitiesAndIntent(
                metrics=main_entities.get("metrics", ["data"]),
                dimensions=main_entities.get("dimensions", []),
                filters=main_entities.get("filters", []),
                primary_goal=main_entities.get("primary_goal", "retrieve"),
                confidence=main_entities.get("confidence", 0.5)
            )
            
            # If complex, second LLM call - decompose the query
            if complexity_analysis["is_complex"]:
                decomposition_prompt = f"""
                Decompose this complex query:
                Query: "{enriched_query}"
                Database ID: {input_data.database_id}
                
                Break down the query into logical parts that can be executed sequentially.
                Identify dependencies between parts.
                Provide assembly instructions for combining results.
                
                Return decomposition in XML format as specified.
                """
                
                if input_data.database_schema:
                    decomposition_prompt += f"\n\nDatabase Schema:\n{input_data.database_schema}"
                
                # Run the agent to get XML decomposition
                decomposition_result_str = await self.agent.arun(task=decomposition_prompt)
                
                # Use the parsing tool to extract structured data
                parsed_decomposition_str = await self.agent.arun(
                    task=f"Parse this XML decomposition response:\n{decomposition_result_str}"
                )
                decomposition_result = json.loads(parsed_decomposition_str)
                
                # Convert parsed parts to QueryPart objects
                query_parts = []
                for part_data in decomposition_result.get("parts", []):
                    entities = ExtractedEntitiesAndIntent(
                        metrics=part_data.get("metrics", []),
                        dimensions=part_data.get("dimensions", []),
                        filters=part_data.get("filters", []),
                        primary_goal=part_data.get("primary_goal", "retrieve"),
                        confidence=0.8
                    )
                    
                    query_parts.append(QueryPart(
                        part_id=part_data.get("part_id", str(uuid.uuid4())),
                        processed_natural_language=part_data.get("sub_query", ""),
                        extracted_entities_and_intent=entities,
                        dependencies=part_data.get("dependencies", []),
                        complexity_level="complex"
                    ))
                
                assembly_instructions = decomposition_result.get("assembly_instructions", "")
                query_type = "Multi_Part_Query"
            else:
                # Single query part
                query_parts = [
                    QueryPart(
                        part_id=str(uuid.uuid4()),
                        processed_natural_language=enriched_query,
                        extracted_entities_and_intent=main_entities_and_intent,
                        dependencies=[],
                        complexity_level="simple"
                    )
                ]
                assembly_instructions = None
                query_type = "Single_Query"
            
            # Create query plan
            query_plan = QueryPlan(
                type=query_type,
                parts=query_parts,
                assembly_instructions_for_multi_part=assembly_instructions
            )
            
            # Create output
            output = QueryAnalysisOutput(
                query_plan=query_plan,
                processed_natural_language=enriched_query,
                extracted_entities_and_intent=main_entities_and_intent,
                overall_query_understanding_summary=self._create_summary(
                    input_data.natural_language_query, 
                    enriched_query, 
                    query_plan
                ),
                confidence_score=complexity_analysis["confidence"]
            )
            
            return output
            
        except Exception as e:
            logger.error(f"Query analysis failed: {str(e)}", exc_info=True)
            raise QueryAnalysisError(f"Failed to analyze query: {str(e)}")
    
    def _create_summary(self, original: str, enriched: str, plan: QueryPlan) -> str:
        """Create a summary of the query understanding."""
        summary_parts = []
        
        summary_parts.append(f"Query interpreted as {plan.type}")
        
        if original != enriched:
            summary_parts.append("Query was enriched with domain knowledge")
        
        if plan.type == "Multi_Part_Query":
            summary_parts.append(f"Decomposed into {len(plan.parts)} parts with dependencies")
        
        goals = set(part.extracted_entities_and_intent.primary_goal for part in plan.parts)
        summary_parts.append(f"Primary goals: {', '.join(goals)}")
        
        return ". ".join(summary_parts) + "."
    
    async def query(self, task: str) -> None:
        """Run a query against the analyzer agent."""
        await Console(self.agent.run_stream(task=task))
    
    async def close(self) -> None:
        """Close the model client connection."""
        await self.model_client.close()

In [6]:
# Initialize configuration
config = {
    'model': 'gpt-4o',
    'temperature': 0.7,
    'max_retry': 3
}

# Create the query analyzer agent with config
query_analyzer = QueryAnalyzerAgent(config=config)

## Test the Query Analyzer Agent

In [ ]:
# Test with a simple query using the new interface
async def test_simple_query():
    query_input = QueryAnalysisInput(
        natural_language_query="Show all schools in Alameda county",
        database_id="california_schools",
        database_schema=None  # Will use default from SCHEMAS
    )
    
    try:
        result = await query_analyzer.analyze(query_input)
        
        print(f"Query Type: {result.query_plan.type}")
        print(f"Processed Query: {result.processed_natural_language}")
        print(f"Confidence: {result.confidence_score}")
        print(f"Summary: {result.overall_query_understanding_summary}")
        print("\nExtracted Entities:")
        print(f"  Metrics: {result.extracted_entities_and_intent.metrics}")
        print(f"  Dimensions: {result.extracted_entities_and_intent.dimensions}")
        print(f"  Filters: {result.extracted_entities_and_intent.filters}")
        print(f"  Primary Goal: {result.extracted_entities_and_intent.primary_goal}")
        
        if result.query_plan.type == "Single_Query":
            part = result.query_plan.parts[0]
            print(f"\nQuery Part ID: {part.part_id}")
            print(f"Natural Language: {part.processed_natural_language}")
            
        return result
    except QueryAnalysisError as e:
        print(f"Analysis failed: {e}")
        return None

# Run the test
result = await test_simple_query()

### Test with Simple Query

In [ ]:
# Test with a complex query involving average comparison
async def test_complex_query():
    query_input = QueryAnalysisInput(
        natural_language_query="List school names of charter schools with an SAT excellence rate over the average",
        database_id="california_schools",
        database_schema=None
    )
    
    try:
        result = await query_analyzer.analyze(query_input)
        
        print(f"Query Type: {result.query_plan.type}")
        print(f"Processed Query: {result.processed_natural_language}")
        print(f"Confidence: {result.confidence_score}")
        print(f"Summary: {result.overall_query_understanding_summary}")
        
        if result.query_plan.type == "Multi_Part_Query":
            print(f"\nQuery decomposed into {len(result.query_plan.parts)} parts:")
            for i, part in enumerate(result.query_plan.parts, 1):
                print(f"\nPart {i} (ID: {part.part_id}):")
                print(f"  Natural Language: {part.processed_natural_language}")
                print(f"  Metrics: {part.extracted_entities_and_intent.metrics}")
                print(f"  Primary Goal: {part.extracted_entities_and_intent.primary_goal}")
                print(f"  Dependencies: {part.dependencies}")
            
            if result.query_plan.assembly_instructions_for_multi_part:
                print(f"\nAssembly Instructions: {result.query_plan.assembly_instructions_for_multi_part}")
        
        return result
    except QueryAnalysisError as e:
        print(f"Analysis failed: {e}")
        return None

# Run the test
complex_result = await test_complex_query()

### Test with Complex Query

In [ ]:
# Test with a multi-step query
async def test_multi_step_query():
    query_input = QueryAnalysisInput(
        natural_language_query="What is the gender of the youngest client who opened account in the lowest average salary branch?",
        database_id="financial",
        database_schema=None
    )
    
    try:
        result = await query_analyzer.analyze(query_input)
        
        print(f"Query Type: {result.query_plan.type}")
        print(f"Confidence: {result.confidence_score}")
        print(f"Summary: {result.overall_query_understanding_summary}")
        
        print("\nMain Query Analysis:")
        print(f"  Processed: {result.processed_natural_language}")
        print(f"  Metrics: {result.extracted_entities_and_intent.metrics}")
        print(f"  Dimensions: {result.extracted_entities_and_intent.dimensions}")
        
        if result.query_plan.type == "Multi_Part_Query":
            print(f"\nDecomposed into {len(result.query_plan.parts)} parts:")
            for i, part in enumerate(result.query_plan.parts, 1):
                print(f"\nPart {i} (ID: {part.part_id}):")
                print(f"  Natural Language: {part.processed_natural_language}")
                print(f"  Primary Goal: {part.extracted_entities_and_intent.primary_goal}")
                if part.dependencies:
                    print(f"  Depends on: {part.dependencies}")
        
        return result
    except QueryAnalysisError as e:
        print(f"Analysis failed: {e}")
        return None

# Run the test
multi_result = await test_multi_step_query()

### Test with Multi-Step Query

### Test with Built-in Domain Knowledge Enrichment

### Test with Domain Knowledge Enrichment

In [ ]:
# Test with a complex aggregation query
async def test_complex_aggregation():
    """Test a query requiring complex aggregation and grouping."""
    query_input = QueryAnalysisInput(
        natural_language_query="For each county with more than 100 schools, show the average SAT math score and the percentage of charter schools",
        database_id="california_schools",
        database_schema=None
    )
    
    try:
        result = await query_analyzer.analyze(query_input)
        
        print(f"Query Type: {result.query_plan.type}")
        print(f"Confidence: {result.confidence_score}")
        print(f"Summary: {result.overall_query_understanding_summary}")
        
        print("\nMain Analysis:")
        entities = result.extracted_entities_and_intent
        print(f"  Metrics: {entities.metrics}")
        print(f"  Dimensions: {entities.dimensions}")
        print(f"  Filters: {entities.filters}")
        print(f"  Primary Goal: {entities.primary_goal}")
        
        if result.query_plan.parts:
            print(f"\nQuery Parts ({len(result.query_plan.parts)}):")
            for i, part in enumerate(result.query_plan.parts, 1):
                print(f"\nPart {i}:")
                print(f"  Description: {part.processed_natural_language}")
                part_entities = part.extracted_entities_and_intent
                print(f"  Metrics: {part_entities.metrics}")
                print(f"  Primary Goal: {part_entities.primary_goal}")
                print(f"  Complexity: {part.complexity_level}")
        
        return result
    except QueryAnalysisError as e:
        print(f"Analysis failed: {e}")
        return None

# Run the test
aggregation_result = await test_complex_aggregation()

### Test with Complex Aggregation Query

## Advanced Example: Using LLM's Automatic Understanding

## Advanced Example: Using Custom Domain Profile

In [ ]:
# Test direct function call with all parameters
async def test_direct_analysis():
    """Test direct analysis with full control over parameters."""
    query_input = QueryAnalysisInput(
        natural_language_query="Find all transactions over $500 from accounts opened in the last year",
        database_id="financial",
        database_schema=None,
        database_profile=None,  # Will use default
        conversation_history=None
    )
    
    try:
        # Direct analysis call
        result = await query_analyzer.analyze(query_input)
        
        print("Direct Analysis Result:")
        print(f"Query Type: {result.query_plan.type}")
        print(f"Confidence: {result.confidence_score}")
        print(f"Summary: {result.overall_query_understanding_summary}")
        
        # Display the analysis results
        print("\nMain Query Analysis:")
        main_entities = result.extracted_entities_and_intent
        print(f"  Metrics: {main_entities.metrics}")
        print(f"  Dimensions: {main_entities.dimensions}")
        print(f"  Filters: {main_entities.filters}")
        print(f"  Primary Goal: {main_entities.primary_goal}")
        
        print(f"\nQuery Parts ({len(result.query_plan.parts)}):")
        for i, part in enumerate(result.query_plan.parts, 1):
            print(f"\nPart {i}:")
            print(f"  ID: {part.part_id}")
            print(f"  Natural Language: {part.processed_natural_language}")
            print(f"  Primary Goal: {part.extracted_entities_and_intent.primary_goal}")
            if part.dependencies:
                print(f"  Dependencies: {part.dependencies}")
        
        return result
    except QueryAnalysisError as e:
        print(f"Analysis failed: {e}")
        return None

# Run the test
direct_result = await test_direct_analysis()

## Test Direct Function Call

In [ ]:
# Test XML parsing capabilities
async def test_xml_parsing():
    """Test the XML parsing mechanism with a complex query that should trigger decomposition."""
    query_input = QueryAnalysisInput(
        natural_language_query="Which districts have charter schools with math scores above the district average and also show the percentage of students eligible for free meals?", 
        database_id="california_schools",
        database_schema=None
    )
    
    try:
        result = await query_analyzer.analyze(query_input)
        
        print("=== XML Parsing Test Results ===")
        print(f"Query Type: {result.query_plan.type}")
        print(f"Confidence: {result.confidence_score}")
        
        if result.query_plan.type == "Multi_Part_Query":
            print(f"\nQuery was successfully decomposed into {len(result.query_plan.parts)} parts:")
            
            for i, part in enumerate(result.query_plan.parts, 1):
                print(f"\n--- Part {i} ---")
                print(f"ID: {part.part_id}")
                print(f"Query: {part.processed_natural_language}")
                print(f"Metrics: {part.extracted_entities_and_intent.metrics}")
                print(f"Dimensions: {part.extracted_entities_and_intent.dimensions}")
                print(f"Filters: {part.extracted_entities_and_intent.filters}")
                print(f"Primary Goal: {part.extracted_entities_and_intent.primary_goal}")
                print(f"Dependencies: {part.dependencies}")
                print(f"Confidence: {part.extracted_entities_and_intent.confidence}")
            
            if result.query_plan.assembly_instructions_for_multi_part:
                print(f"\nAssembly Instructions:\n{result.query_plan.assembly_instructions_for_multi_part}")
        else:
            print("\nQuery treated as single query:")
            part = result.query_plan.parts[0]
            print(f"Query: {part.processed_natural_language}")
            print(f"Metrics: {part.extracted_entities_and_intent.metrics}")
            print(f"Primary Goal: {part.extracted_entities_and_intent.primary_goal}")
        
        return result
    except Exception as e:
        print(f"Test failed: {type(e).__name__}: {e}")
        return None

# Run the XML parsing test
xml_result = await test_xml_parsing()